<img src='./img/LogoWekeo_Copernicus_RGB_0.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='20%'></img>

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./20_sentinel3_olci_l1_retrieve.ipynb"><< 20 - Sentinel-3 OLCI Level 1B - Retrieve</a><span style="float:right;"><a href="./30_cams_eac4_retrieve.ipynb">30 - CAMS Glocal Reanalysis EAC4 - Retrieve >></a></span>

<div class="alert alert-block alert-warning">
<b>LOAD, BROWSE AND VISUALIZE</b></div>

# Sentinel-3 OLCI Level 1B Full Resolution data

Sentinel-3 OLCI Level 1B data products are available as Full (`OL_1_EFR`) and Reduced Resolution (`OL_1_ERR`) data files. The following notebook shows you how `OL_1_EFR` data are structured and how you can generate a RGB (Red, Green, Blue) True- and False-Color image.

True-Color images combine reflectances from the red, green and blue light and resulting images look like real-world images. False-Color composites combine reflectances from the visible (light) and non-visible wavelengths. Depending on the application, specific features might be better identifiable as a false-color composite. 

Sentinel-3 OLCI data products are disseminated as `.zip archives`, containing data files in `NetCDF` format.

Find more information on the OLCI data products in the [Sentinel-3 OLCI User Guide](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-3-olci).

### Outline
* [1 - Load and browse Sentinel-3 OLCI Level 1B data](#load_browse)
* [2 - Load OLCI channel information as xarray](#channel_info)
* [3 - Load OLCI geographic coordinates as xarray](#load_geo_coords)
* [4 - Select OLCI channels for a RGB image](#channel_select)
* [5 - Advanced image processing - Normalization and histogram equalization](#image_processing)
* [6 - Visualize a georeferenced RGB True Color image with pcolormesh](#visualize_tc)
* [7 - Create a False Color RGB image and plot it as georeferenced image ](#false_color)

<hr>

#### Load required libraries

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
from skimage import exposure

import glob
import os
import subprocess
import shlex
import zipfile

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

<hr>

### Decompressing Sentinel-3 OLCI Level 1B Full Resolution data

The Sentinel-3 OLCI Level 1B data could be not available on the machine. In this case there is the method to download them to [20 - Sentinel-3 OLCI Level 1B Retrieve](./20_sentinel3_olci_l1_retrieve.ipynb).

Also the user must extract the data before load them, if they are provided as a zip archive. If you have an already decompressed data this step is not necessary and the data can be directly loaded.

In [ ]:
#rename S3 file
download_dir_path = os.getcwd()

for item in os.listdir(download_dir_path): 
    if not os.path.isdir(item):
        cmd = shlex.split('file --mime-type {0}'.format(item))
        result = subprocess.check_output(cmd)
        mime_type = result.split()[-1]
        if item.startswith('S3B_OL_1_') and not item.endswith('.zip') and mime_type == b'application/zip':
            os.rename(os.path.join(download_dir_path, item), os.path.join(download_dir_path, item +".zip"))

In case you have permission issues when renaming the archive with the code above, please manually do that by right-clicking the file, then rename, then add .zip to the end of the filename.

For example:
- From: S3B_OL_1_EFR____20211103T225931_20211103T230231_20211105T023456_0179_059_001_3600_MAR_O_NT_002   
- To: S3B_OL_1_EFR____20211103T225931_20211103T230231_20211105T023456_0179_059_001_3600_MAR_O_NT_002.zip

In [ ]:
extension = ".zip"

for item in os.listdir(download_dir_path): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.join(download_dir_path, item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(download_dir_path) # extract file to dir
        zip_ref.close() # close file
        

## <a id='load_browse'></a>Load and browse Sentinel-3 OLCI Level 1B Full Resolution data

Sentinel-3 OLCI Level 1B data are dissiminated as `.zip archives` when downloaded. The unzipped folder contains 29 data files in `netCDF` format. Data for each channel is stored in a single `netCDF` file. Additionally, you get information on `qualityFlags`, `time_coordinates` or `geo_coordinates`.

You can see the names of the 29 data files by looping through the data directory. OLCI has 21 spectral bands representing reflectances from optical to the near-infrared wavelength. Each spectral band is represented in a channel. The files that end with `_radiance.nc` contain the reflectance information for each channel.

In [ ]:
olci_dir = './S3B_OL_1_EFR____20211103T225931_20211103T230231_20211105T023456_0179_059_001_3600_MAR_O_NT_002.SEN3/'
for i in glob.glob(olci_dir+'*.nc'):
    tmp = i.split('/')
    print(tmp[2])

## <a id='channel_info'></a>Load OLCI channel information as `xarray`

#### Load one single channel

As a first step, you can load one channel (spectral band) with xarray's function `xr.open_dataset()`. This will help you to understand how the data is structured.
You see that the data for each channel is a two dimensional data array, with `rows` and `columns` as dimensions. 

In [ ]:
olci_xr = xr.open_dataset(olci_dir+'Oa01_radiance.nc')
olci_xr

#### Load all channel information into one xarray object

The next step is to bring the 21 data files of the spectral bands (channels) together into one `xarray` data object. You can do this with xarray's function `open_mfdataset`. Since all channel files have the same data structure, you can combine all channels by coordinates using the keyword argument `by_coords`. The result is a `xarray` object with 21 data variables. Each channel is represented as a data variable.

In [ ]:
olci_xr_mf = xr.open_mfdataset(olci_dir+'*_radiance.nc', combine='by_coords')
olci_xr_mf

#### Example to plot one channel

To get an impression of one spectral band, you can simply plot one channel of the xarray object with xarray's function  `.plot.imshow()`. Let us plot channel 8 `Oa08_radiance` as an example. The example shows that you can easily visualize the the spectral band information, but without geographic information.

In [ ]:
fig = plt.figure(figsize=(10,6))
olci_xr_mf['Oa08_radiance'].plot.imshow()

## <a id='load_geo_coords'></a>Load OLCI geographic coordinates as `xarray`

If you want to georeference your image, you have to load the geographic coordinates file. You can load it as xarray with the function `xr.open_dataset()`. The file is called `geo_coordinates.nc`. You see that the file contains three variables: `latitude`, `longitude` and `altitude`. Latitude and longitude are the geographical coordinates. Altitude is the DEM corrected altitude of the 'Top-of-the-atmosphere' reflectance.

Let's store the latitude and longitude data as `lat` and `lon` variables respectively.

In [ ]:
olci_geo_coords = xr.open_dataset(olci_dir+'geo_coordinates.nc')
olci_geo_coords

In [ ]:
lat = olci_geo_coords.latitude.data
lon = olci_geo_coords.longitude.data

## <a id='channel_select'></a>Select OLCI channels for a True-Color RGB image

True-Color images combine reflectances from the `red` (B04), `green` (B06) and `blue` (B04) light and resulting images look like real-world images, representing the Earth as humans see it naturally.


 The channel combination for a True-Color RGB (Red, Green, Blue) image of Sentinel-3 OLCI Level 1B data could be:
- Red: `Oa08_radiance`
- Green: `Oa06_radiance`
- Blue: `Oa04_radiance`

Alternatively, you can also try the combination `red` (B09), `green` (B06) and `blue` (B04).

Let us define a function called [select_channels_for_rgb](./functions.ipynb#rgb_channels), which makes the channel selection more flexible. The function returns the three bands individually.

In [ ]:
def select_channels_for_rgb(xarray, red_channel, green_channel, blue_channel):
    """ 
    Selects the channels / bands of a multi-dimensional xarray for red, green and blue composites.
    
    Parameters:
        xarray(xarray Dataset): xarray Dataset object that stores the different channels / bands.
        red_channel(str): Name of red channel to be selected
        green_channel(str): Name of green channel to be selected
        blue_channel(str): Name of blue channel to be selected

    Returns:
        Three xarray DataArray objects with selected channels / bands
    """  
    return xarray[red_channel], xarray[green_channel], xarray[blue_channel]

In [ ]:
red, green, blue = select_channels_for_rgb(olci_xr_mf, 'Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance')
red

## <a id='image_processing'></a>Advanced image processing - Normalization and histogram equalization

#### Normalization

A common operation in image processing is the normalization of data values. Normalization changes the range of pixel intensity and can improve the contrast.
Let us define a function called [normalize](./functions.ipynb#normalize), which normalizes a numpy array into a scale between 0.0 and 1.0.

In [ ]:
def normalize(array):
    """ 
    Normalizes a numpy array / xarray DataArray object value to values between 0 and 1.
    
    Parameters:
        xarray(numpy array or xarray DataArray): xarray DataArray or numpy array object.

    Returns:
        Normalized array
    """ 
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

You can now apply the function [normalize](./functions.ipynb#normalize) to each RGB channel, **red**, **green** and **blue**. At the end, you can bring the three channels together into one `rgb` array with `numpy.dstack`. By verifying the shape of the resulting array, you see that the resulting `rgb` array has three dimensions, `rows`, `columns` and `number of channels`.

In [ ]:
redn = normalize(red)
greenn = normalize(green)
bluen = normalize(blue)

In [ ]:
rgb = np.dstack((redn, greenn, bluen))
rgb.shape

You can now plot the **rgb** image to see if maybe further image processing methods could be applied. If you visualize the image with `plt.imshow()`, you see that the constrast is not very strong. A further step is to sharpen the contrast with the help of a **Histogram equalization**.

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.imshow(rgb)

#### Histogram equalization

Histogram equalization is a method in image processing that adjusts the contrast using the image's histogram. Python's `skikit-learn` library has useful tools to make a histogram equalization. The `skimage` library provides a function `exposure.equalize_adaphtist()` which can be applied to the **rgb** data array.

In [ ]:
rgb = exposure.equalize_adapthist(rgb)

You can again plot the rgb image with `plt.imshow()`. You can see that the histogram equalization improved the contrast of the image.

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.imshow(rgb)

## <a id='visualize_tc'></a>Visualize a georeferenced RGB True-Color image with `pcolormesh`

The final step is to georeference the `RGB` image. Therefore, you need the extracted `lat` and `lon` information extracted above.

Let us plot the image with matplotlib's `pcolormesh` function. To be able to visualize the array as RGB True Color images, you have to map first the rgb array to a colour array.

In [ ]:
mesh_rgb = rgb[:,:-1,:]
colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)

The last step is to visualize the color array and add additional information, e.g. gridlines and coastlines. As you want to reuse the plotting code again, you can define a function and call it [visualize_s3_pcolormesh](./functions.ipynb#visualize_s3_pcolormesh).

In [ ]:
def visualize_s3_pcolormesh(color_array, array, latitude, longitude, title):
    """ 
    Visualizes a numpy array (Sentinel-3 data) with matplotlib's 'pcolormesh' function as RGB image.
    
    Parameters:
        color_array (numpy.ndarray): a numpy.ndarray holding the color tuples to be used to colorize the image
        array (xarray.DataArray): an xarray.DataArray that provides the shape of the resulting array to be plotted
        longitude (numpy.ndarray): array with longitude values
        latitude (numpy.ndarray) : array with latitude values
        title (str): title of the resulting plot
    """
    fig=plt.figure(figsize=(20, 12))

    ax=plt.axes(projection=ccrs.Mercator())
    ax.coastlines()

    gl = ax.gridlines(draw_labels=True, linestyle='--')
    gl.xformatter=LONGITUDE_FORMATTER
    gl.yformatter=LATITUDE_FORMATTER
    gl.xlabel_style={'size':14}
    gl.ylabel_style={'size':14}

    img1 = plt.pcolormesh(longitude, latitude, array*np.nan, color=color_array,
                          clip_on = True,
                          edgecolors=None,
                          zorder=0,
                          transform=ccrs.PlateCarree())
    ax.set_title(title, fontsize=20, pad=40.0)
    plt.show()

As color_array, we use the `colorTuple` array and as array we use the shape of e.g. the `red` channel information.

In [ ]:
visualize_s3_pcolormesh(colorTuple,red, lat, lon, 'Sentinel-3 OLCI Level-1 True Color RGB - "29 December 2019"')

<br>

## <a id='false_color'></a>Create a False Color RGB image and plot it as georeferenced image 

Instead of a true-color RGB image, a false-color RGB image can be created if specific phenomena should be highlighted. A channel combination to highlight smoke and fires from Sentinel-3 OLCI Level 1B data could be:
- Red: `Oa17_radiance`
- Green: `Oa05_radiance`
- Blue: `Oa02_radiance`

This selection identifies easier burnt areas and wildfires. The combination highlights healthy vegetation as red and burnt areas as black.

Let's repeat the steps from above to visualize the image as False Color RGB composite.

#### Select RGB channels

The first step is to select the channels `Oa17_radiance`, `Oa05_radiance` and `Oa02_radiance` as red, green and blue channels respectively.

In [ ]:
red_fc, green_fc, blue_fc = select_channels_for_rgb(olci_xr_mf, 'Oa17_radiance', 'Oa05_radiance', 'Oa02_radiance')

#### Normalize

You also want to normalize these channels and stack them afterwards into a three-dimensional array.

In [ ]:
redn_fc = normalize(red_fc)
greenn_fc = normalize(green_fc)
bluen_fc = normalize(blue_fc)

In [ ]:
rgb_fc = np.dstack((redn_fc, greenn_fc, bluen_fc))

#### Histogram equalization

Apply histogram equalization.

In [ ]:
rgb_fc = exposure.equalize_adapthist(rgb_fc)

Let's have a sneak-peak to the un-georefrenced image with `plt.imshow()`

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.imshow(rgb_fc)

#### Visualize with function `visualize_s3_pcolormesh`

First, map the stacked rgb array to a colour array and call it **colorTuple**.

In [ ]:
mesh_rgb = rgb_fc[:, :-1, :]
colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)
colorTuple = np.insert(colorTuple, 3, 1.0, axis=1)

As a final step, visualize the geo-referenced false-colour RGB composite with the function [visualize_s3_pcolormesh](./functions.ipynb#visualize_s3_pcolormesh).

In [ ]:
visualize_s3_pcolormesh(colorTuple, red_fc, lat, lon, 'Sentinel-3 OLCI Level-1 False Color RGB - "29 December 2019"')

<br>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./20_sentinel3_olci_l1_retrieve.ipynb"><< 20 - Sentinel-3 OLCI Level 1B - Retrieve</a><span style="float:right;"><a href="./30_cams_eac4_retrieve.ipynb">30 - CAMS Glocal Reanalysis EAC4 - Retrieve >></a></span>

<hr>

<img src='./img/all_partners_wekeo.png' alt='Logo EU Copernicus EUMETSAT' align='right' width='100%'></img>